In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/CS\ 224N/CS\ 224N\ Project
! ls # verify that you are in the right directory

Mounted at /content/drive
/content/drive/MyDrive/CS 224N/CS 224N Project
'224N Project Brainstorm.gdoc'				 csvs
'224N Project Helpful Tutorials.gdoc'			 dataset_agg.ipynb
'224N Project Milestone Notes.gdoc'			'first proposal OLD'
 aita_clean.csv						 gpt2-baseline.ipynb
 aita_comments.csv					'Reddit Scraper.ipynb'
 bert-baseline						 T5-baseline.ipynb
'Copy of fine_tune_bart_summarization_two_langs.ipynb'


In [ ]:
import os
import pandas as pd
path = os.listdir('csvs')
print(path)

['comments_batch_8273.csv', 'comments_batch_11473.csv', 'comments_batch_4067.csv', 'comments_batch_15737.csv', 'comments_batch_19402.csv', 'comments_batch_21869.csv', 'comments_batch_60700.csv', 'comments_batch_62058.csv', 'comments_batch_24666.csv', 'comments_batch_62270.csv', 'comments_batch_63000.csv', 'comments_batch_25587.csv', 'comments_batch_63037.csv', 'comments_batch_64000.csv', 'comments_batch_65000.csv', 'comments_batch_26957.csv', 'comments_batch_33231.csv', 'comments_batch_65660.csv', 'comments_batch_66000.csv', 'comments_batch_27485.csv', 'comments_batch_66593.csv', 'comments_batch_33688.csv', 'comments_batch_67000.csv', 'comments_batch_68000.csv', 'comments_batch_69000.csv', 'comments_batch_69361.csv', 'comments_batch_70000.csv', 'comments_batch_37064.csv', 'comments_batch_71000.csv', 'comments_batch_72000.csv', 'comments_batch_73000.csv', 'comments_batch_74000.csv', 'comments_batch_75000.csv', 'comments_batch_76000.csv', 'comments_batch_77000.csv', 'comments_batch_78000

In [ ]:
df = pd.DataFrame()
# %cd csvs
# ! ls
for i, f in enumerate(path):
  df_new = pd.read_csv(f)
  # df_new = pd.read_csv(os.path.join(os.getcwd(), f))

  df = pd.concat([df, df_new])

# remove duplicate ids
df = df.drop_duplicates(subset=['id'])
print(df.head())

/content/drive/MyDrive/CS 224N/CS 224N Project/csvs
comments_batch_11473.csv  comments_batch_63037.csv  comments_batch_80000.csv
comments_batch_15737.csv  comments_batch_64000.csv  comments_batch_81000.csv
comments_batch_19402.csv  comments_batch_65000.csv  comments_batch_82000.csv
comments_batch_21869.csv  comments_batch_65660.csv  comments_batch_8273.csv
comments_batch_24666.csv  comments_batch_66000.csv  comments_batch_83000.csv
comments_batch_25587.csv  comments_batch_66593.csv  comments_batch_84000.csv
comments_batch_26957.csv  comments_batch_67000.csv  comments_batch_85000.csv
comments_batch_27485.csv  comments_batch_68000.csv  comments_batch_86000.csv
comments_batch_32000.csv  comments_batch_69000.csv  comments_batch_87000.csv
comments_batch_33231.csv  comments_batch_69361.csv  comments_batch_88000.csv
comments_batch_33688.csv  comments_batch_70000.csv  comments_batch_89000.csv
comments_batch_37064.csv  comments_batch_71000.csv  comments_batch_90000.csv
comments_batch_4067.csv  

In [ ]:
print(len(df))

87215


In [ ]:
# Remove any elements that are [deleted] from the dataframe
# also remove [removed] posts --> these are posts the mods deleted
df = df[~df['comments'].str.contains('\[deleted\]')]
print(len(df))
df = df[~df['comments'].str.contains('\[removed\]')]
print(len(df))


84197
83929


In [ ]:
# %cd ..
df_clean = pd.read_csv("aita_clean.csv")
print(len(df_clean))
df_combined = pd.merge(df, df_clean, on='id', how='inner')
print(len(df_combined))

/content/drive/MyDrive/CS 224N/CS 224N Project
97628
83929


In [ ]:
# save the new data frame in a file called "aita_comments.csv"
df_combined.to_csv("/content/drive/MyDrive/CS 224N/CS 224N Project/aita_comments.csv", index=False) 

Conduct final preprocessing on our dataset to structure it appropriately for our generative model

In [2]:
import os
import pandas as pd

In [27]:
df = pd.read_csv("aita_comments.csv")


In [28]:
df["text"] = df["title"].astype(str) +". "+ df["body"].astype(str)

# create a one hot vector for the verdicts
# ["asshole", "everyone sucks", "no assholes here", "not the asshole"]
# this gets changed to ["v_yta", "v_esh", "v_nah", "v_nta"] --> v stands for verdict and the verdict after is abbreviated
one_hot = pd.get_dummies(df['verdict'])
one_hot = one_hot.rename(columns = {'asshole': 'v_yta', 'everyone sucks': 'v_esh', 'no assholes here': 'v_nah', 'not the asshole': 'v_nta'})

df = pd.concat([df, one_hot], axis=1)

df = df[["text", "comments", "is_asshole", "v_yta", "v_esh", "v_nah", "v_nta"]] # drop unused columns 


In [29]:
# normalize text length based on average and mean length of posts + title combo
avg_words = df['comments'].str.split().str.len().mean()
avg_words_text = df['text'].str.split().str.len().mean()

print(f'Average comment length is: ${avg_words}')
print(f'Average text length is: ${avg_words_text}')

median_words = df['comments'].str.split().str.len().median()
median_words_text = df['text'].str.split().str.len().median()
print(f'Median comment length is: ${median_words}')
print(f'Median text length is: ${median_words_text}')


Average comment length is: $49.38626696374316
Average text length is: $348.63994566836254
Median comment length is: $38.0
Median text length is: $320.0


In [30]:
cutoff = 350
num_long_comments = (df['text'].apply(lambda x: len(x.split(' '))) > cutoff).sum()
print(f'The number of comments that are truncated with cutoff: {cutoff} and its percentage of the total dataset')
print(num_long_comments)
print(num_long_comments / len(df))
# if we were to set our cap at 400, these are the number of comments that'd be 
# affected. We also display the percent of the total that will get truncated
cutoff = 400
num_long_comments = (df['text'].apply(lambda x: len(x.split())) > cutoff).sum()
print(f'The number of comments that are truncated with cutoff: {cutoff} and its percentage of the total dataset')
print(num_long_comments)
print(num_long_comments / len(df))

cutoff = 450
num_long_comments = (df['text'].apply(lambda x: len(x.split())) > cutoff).sum()
print(f'The number of comments that are truncated with cutoff: {cutoff} and its percentage of the total dataset')
print(num_long_comments)
print(num_long_comments / len(df))

cutoff = 500
num_long_comments = (df['text'].apply(lambda x: len(x.split())) > cutoff).sum()
print(f'The number of comments that are truncated with cutoff: {cutoff} and its percentage of the total dataset')
print(num_long_comments)
print(num_long_comments / len(df))

cutoff = 550
num_long_comments = (df['text'].apply(lambda x: len(x.split())) > cutoff).sum()
print(f'The number of comments that are truncated with cutoff: {cutoff} and its percentage of the total dataset')
print(num_long_comments)
print(num_long_comments / len(df))

cutoff = 600
num_long_comments = (df['text'].apply(lambda x: len(x.split())) > cutoff).sum()
print(f'The number of comments that are truncated with cutoff: {cutoff} and its percentage of the total dataset')
print(num_long_comments)
print(num_long_comments / len(df))

The number of comments that are truncated with cutoff: 350 and its percentage of the total dataset
36335
0.43292544889132484
The number of comments that are truncated with cutoff: 400 and its percentage of the total dataset
29045
0.34606631795922743
The number of comments that are truncated with cutoff: 450 and its percentage of the total dataset
22501
0.2680956522775203
The number of comments that are truncated with cutoff: 500 and its percentage of the total dataset
17035
0.20296917632761025
The number of comments that are truncated with cutoff: 550 and its percentage of the total dataset
11847
0.14115502388923973
The number of comments that are truncated with cutoff: 600 and its percentage of the total dataset
4981
0.059347782053878875


We chose 508 as the cutoff because the T5 baseline can support at most 512 words. We wanted to include the last 4 words as the prompt "Am I the Asshole" because our research found that including a prompt at the end of the text sample greatly helped generative models generate text. 

508 is also a healthy cutoff because less than 1/5 of our training data will be truncated, it's 19.335% to be exact. 


In [31]:
cutoff = 508
num_long_comments = (df['text'].apply(lambda x: len(x.split())) > cutoff).sum()
print(f'The number of comments that are truncated with cutoff: {cutoff} and its percentage of the total dataset')
print(num_long_comments)
print(num_long_comments / len(df))

# Truncate all the strings in the dataset longer than cutoff 
df = df[df['text'].apply(lambda x: len(x.split(' ')) < cutoff)]

# add a prompt to help generative model with prompt engineering 
question = ". Am I the asshole?"
df["text"] = df["text"].astype(str) + question

avg_words_text_final = df['text'].str.split().str.len().mean()
print(f'Average text length is: ${avg_words_text_final}')

median_words_text_final = df['text'].str.split().str.len().median()
print(f'Median text length is: ${median_words_text_final}')


The number of comments that are truncated with cutoff: 508 and its percentage of the total dataset
16228
0.1933539062779254


<ipython-input-31-16c128f5d091>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"] = df["text"].astype(str) + question


Average text length is: $287.85849626497327
Median text length is: $281.0


In [32]:
#Split validation and test set
valid_set = df.sample(n = 1000)
df = df.loc[~df.index.isin(valid_set.index)]

#Reset the indexes
valid_set = valid_set.reset_index()
df = df.reset_index()

test_set = df.sample(n = 1000)
df = df.loc[~df.index.isin(test_set.index)]

#Reset the indexes
test_set = test_set.reset_index()
df = df.reset_index()

In [33]:
# save the new data frames "
df.to_csv("/content/drive/MyDrive/CS 224N/CS 224N Project/aita_train_set.csv", index=False) 
valid_set.to_csv("/content/drive/MyDrive/CS 224N/CS 224N Project/aita_valid_set.csv", index=False)
test_set.to_csv("/content/drive/MyDrive/CS 224N/CS 224N Project/aita_test_set.csv", index=False)